## Imports

In [10]:
from base_Imports_and_Functions import *

### Thermals
Thermals_db = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/netcds/Thermals_data_base.nc')

thermals_19 = Thermals_db.where(
    (Thermals_db['dayofflight'] == 19),
    drop = True)

reduced_thermal_19 = thermals_19.where(
    (thermals_19['latitude'] > extent_son[2]) & (thermals_19['latitude'] < extent_son[3]) & 
    (thermals_19['longitude'] > extent_son[0]) & (thermals_19['longitude'] < extent_son[1]),
    drop = True)

reduced_thermal_19_12 = reduced_thermal_19.where(
    (thermals_19['rawtime'] >= 11*60*60) & (thermals_19['rawtime'] <= 12*60*60),
    drop = True)

reduced_thermal_19_15 = reduced_thermal_19.where(
    (thermals_19['rawtime'] >= 14*60*60) & (thermals_19['rawtime'] <= 15*60*60),
    drop = True)

## Comparison product

### Loading data

In [ ]:
# Meso-NH 200 (son)
simu_BLH_19_son = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_19.nc')   
simu_BLH_20_son = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_20.nc')

# Meso-NH 600 (father)
simu_BLH_19_father = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_0_23.nc')   
simu_BLH_20_father = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_20_0_23.nc')

# AROME 1.3 km (grand father)
simu_BLH_19_AROME = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_19.nc')   
simu_BLH_20_AROME = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_20.nc')

### Comparison

In [26]:
@njit()
def compare_ground_BLH(lon_inf,lon_sup,lat_inf,lat_sup,Glide_array,lon_simu,lat_simu,Simu_array):

    res = np.zeros((1,len(lon_inf)))[0]
    res_trace = np.zeros((1,len(lon_inf)))[0]
    res_simu = np.zeros((1,len(lon_inf)))[0]

    for i in range(len(lon_inf)):
        
        extent_i = (lon_inf[i],lon_sup[i],lat_inf[i],lat_sup[i])
        reduced_array = Simu_array[(lon_simu<extent_i[1]) & (lon_simu>extent_i[0]) & (lat_simu<extent_i[3]) & (lat_simu>extent_i[2])]

        if len(reduced_array) != 0 and np.isnan(Glide_array[i]) == False :
            BLH_simu_array = np.max(reduced_array)
            res[i] = Glide_array[i] - BLH_simu_array
            res_trace[i] = Glide_array[i]
            res_simu[i] = BLH_simu_array

        else :
            res[i] = np.nan
            res_trace[i] = np.nan
            res_simu[i] = np.nan
        
    return res,res_trace,res_simu

@njit()
def compare_ground_topo(lon_inf,lon_sup,lat_inf,lat_sup,lon_simu,lat_simu,topo_array):

    mean_ = np.zeros((1,len(lon_inf)))[0]
    hquart_ = np.zeros((1,len(lon_inf)))[0]
    std_ = np.zeros((1,len(lon_inf)))[0]

    for i in range(len(lon_inf)):
        extent_i = (lon_inf[i],lon_sup[i],lat_inf[i],lat_sup[i])
        elevation_i = topo_array[(lon_simu<extent_i[1]) & (lon_simu>extent_i[0]) & (lat_simu<extent_i[3]) & (lat_simu>extent_i[2])]
       
        if len(elevation_i) != 0 :
            mean_[i] = np.nanmean(elevation_i)
            hquart_[i] = np.nanpercentile(elevation_i,90)
            std_[i] = np.nanstd(elevation_i)

        else :
            mean_[i] = np.nan
            hquart_[i] = np.nan
            std_[i] = np.nan
        
    return mean_,hquart_,std_

@njit()
def compare_ground_W_max(lon_inf,lon_sup,lat_inf,lat_sup,Glide_array,lon_simu,lat_simu,Simu_array):
    
    res = np.zeros((1,len(lon_inf)))[0]
    res_trace = np.zeros((1,len(lon_inf)))[0]
    res_simu = np.zeros((1,len(lon_inf)))[0]

    for i in range(len(lon_inf)):
        
        extent_i = (lon_inf[i],lon_sup[i],lat_inf[i],lat_sup[i])
        reduced_array = Simu_array[(lon_simu<extent_i[1]) & (lon_simu>extent_i[0]) & (lat_simu<extent_i[3]) & (lat_simu>extent_i[2])]

        if len(reduced_array) != 0 : 
            BLH_simu_array = np.max(Simu_array[(lon_simu<extent_i[1]) & (lon_simu>extent_i[0]) & (lat_simu<extent_i[3]) & (lat_simu>extent_i[2])])
            res[i] = Glide_array[i] - BLH_simu_array
            res_trace[i] = Glide_array[i]
            res_simu[i] = BLH_simu_array
            
        else :
            res[i] = np.nan
            res_trace[i] = np.nan
            res_simu[i] = np.nan
            
    return res
    
def Comp(day,type,dlat,dlon,Simu_0_23,save_name):

#    Simu_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/'+str(day)+'.nc')'T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_0_23.nc'
    
#    Simu_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/son_19_0_23.nc')
    Glide = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/'+str(day)+'_'+type+'.nc')

#    extent_son = (5.499792, 7.032229, 44.104446, 45.18888) # Maintenant les comparaison se font uniquement sur le fils
    extent_son = (5, 8, 43, 46)
 
    Simu_0_23 = Simu_0_23.where(
        (Simu_0_23['lon'] > extent_son[0]) & (Simu_0_23['lon'] < extent_son[1]) & 
        (Simu_0_23['lat'] > extent_son[2]) & (Simu_0_23['lat'] < extent_son[3]),
        drop = True)


    if type == 'BLH' :

        Glide = Glide.where(
            (Glide['longitude_inf'] > extent_son[0]) & (Glide['longitude_inf'] < extent_son[1]) & 
            (Glide['latitude_inf'] > extent_son[2]) & (Glide['latitude_inf'] < extent_son[3]) &
            (Glide['longitude_sup'] > extent_son[0]) & (Glide['longitude_sup'] < extent_son[1]) & 
            (Glide['latitude_sup'] > extent_son[2]) & (Glide['latitude_sup'] < extent_son[3]),
            drop = True)
    
        bias_i = np.zeros((1,1))[0]
        lon_comp_i = np.zeros((1,1))[0]
        lat_comp_i = np.zeros((1,1))[0]
        mean_topo_i = np.zeros((1,1))[0]
        hquart_topo_i = np.zeros((1,1))[0]
        std_i = np.zeros((1,1))[0]
        heure_i = np.zeros((1,1))[0]
        traces_i = np.zeros((1,1))[0]
        simu_i = np.zeros((1,1))[0]
        
        time_stemps = np.array([[i,(i+1)] for i in range(10,19)])
        for it in tqdm(range(len(time_stemps))):
            time_stemp = time_stemps[it]
    
            Glide_it = Glide.where(
                (Glide['heure'] == time_stemp[0]), # tout mettre en heure
                drop = True)
    
            lon_inf = Glide_it['longitude_inf'].values
            lon_sup = Glide_it['longitude_sup'].values
            lat_inf = Glide_it['latitude_inf'].values
            lat_sup = Glide_it['latitude_sup'].values
            Glide_array = Glide_it['PBLH'].values
            
            Simu_it = Simu_0_23.where(
                (Simu_0_23['heure'] == time_stemp[1]),
                drop = True)

            if len(Simu_it['heure']) != 0 :
    
                lon_simu = Simu_it['lon'].values
                lat_simu = Simu_it['lat'].values
                Simu_array = Simu_it['BLH'].values
                topo_array = Simu_it['ZS'].values

                bias,trace,simu = compare_ground_BLH(lon_inf,lon_sup,lat_inf,lat_sup,Glide_array,lon_simu,lat_simu,Simu_array)
                mean_, percenth, std_ = compare_ground_topo(lon_inf,lon_sup,lat_inf,lat_sup,lon_simu,lat_simu,topo_array)
        
                lon_comp_i = np.concatenate((lon_comp_i,(lon_inf+lon_sup)/2), axis = 0)
                lat_comp_i = np.concatenate((lat_comp_i,(lat_inf+lat_sup)/2), axis = 0)
                bias_i = np.concatenate((bias_i,bias), axis = 0)
                mean_topo_i = np.concatenate((mean_topo_i,mean_), axis = 0)
                hquart_topo_i = np.concatenate((hquart_topo_i,percenth), axis = 0)
                std_i = np.concatenate((std_i,std_), axis = 0)
                heure_i = np.concatenate((heure_i,np.ones((1,len(bias)))[0]*time_stemp[1]), axis = 0)
                traces_i = np.concatenate((traces_i,trace), axis = 0)
                simu_i = np.concatenate((simu_i,simu), axis = 0)

        num_events = len(heure_i)-1
        num_data_points = 7
        Comp_simu_BLH = xr.Dataset(
            {
            'lon_comp': (['event'], lon_comp_i[1:].astype(np.float32)),
            'lat_comp': (['event'], lat_comp_i[1:].astype(np.float32)),
            'bias': (['event'], bias_i[1:].astype(np.float32)),
            'traces': (['event'], traces_i[1:].astype(np.float32)),
            'simu': (['event'], simu_i[1:].astype(np.float32)),
            'mean_topo': (['event'], mean_topo_i[1:].astype(np.float32)),
            'hquart_topo': (['event'], hquart_topo_i[1:].astype(np.float32)),
            'std_topo': (['event'], std_i[1:].astype(np.float32)),
            'heure': (['event'], heure_i[1:].astype(int))
        },
        coords={
            'event': np.arange(num_events)
        },
        )
        
        Comp_simu_BLH.to_netcdf(savenet_path + 'Comp_'+save_name+'_'+type+'_'+'.nc')       

    if type == 'W_max' :

        Glide = Glide.where(
            (Glide['lon'] > extent_son[0]) & (Glide['lon'] < extent_son[1]) & 
            (Glide['lat'] > extent_son[2]) & (Glide['lat'] < extent_son[3]),
            drop = True)
    
        bias_i = np.zeros((1,1))[0]
        lon_comp_i = np.zeros((1,1))[0]
        lat_comp_i = np.zeros((1,1))[0]
        mean_topo_i = np.zeros((1,1))[0]
        hquart_topo_i = np.zeros((1,1))[0]
        std_i = np.zeros((1,1))[0]
        heure_i = np.zeros((1,1))[0]
        traces_i = np.zeros((1,1))[0]
        simu_i = np.zeros((1,1))[0]
        
        time_stemps = np.array([[i,(i+1)] for i in range(10,19)])
        for it in tqdm(range(len(time_stemps))):
            time_stemp = time_stemps[it]
    
            Glide_it = Glide.where(
                (Glide['heure'] == time_stemp[0]),
                drop = True)
    
            lon = Glide_it['lon'].values
            lat = Glide_it['lat'].values
            Glide_array = Glide_it['max_speed'].values
            
            Simu_it = Simu_0_23.where(
                (Simu_0_23['iteration'] == time_stemp[1]-7),
                drop = True)

            if len(Simu_it['iteration']) != 0 :
    
                lon_simu = Simu_it['lon'].values
                lat_simu = Simu_it['lat'].values
                Simu_array = Simu_it['W_max'].values
                topo_array = Simu_it['ZS'].values
    
                lon_inf = lon-dlon
                lon_sup = lon+ dlon
                lat_inf = lat-dlat
                lat_sup = lat+dlat
        
                bias,trace,simu = compare_ground_W_max(lon_inf,lon_sup,lat_inf,lat_sup,Glide_array,lon_simu,lat_simu,Simu_array)
                mean_, percenth, std_ = compare_ground_topo(lon_inf,lon_sup,lat_inf,lat_sup,lon_simu,lat_simu,topo_array)
        
                lon_comp_i = np.concatenate((lon_comp_i,(lon_inf+lon_sup)/2), axis = 0)
                lat_comp_i = np.concatenate((lat_comp_i,(lat_inf+lat_sup)/2), axis = 0)
                bias_i = np.concatenate((bias_i,bias), axis = 0)
                mean_topo_i = np.concatenate((mean_topo_i,mean_), axis = 0)
                hquart_topo_i = np.concatenate((hquart_topo_i,percenth), axis = 0)
                std_i = np.concatenate((std_i,std_), axis = 0)
                heure_i = np.concatenate((heure_i,np.ones((1,len(bias)))[0]*time_stemp[1]), axis = 0)
                traces_i = np.concatenate((traces_i,trace), axis = 0)
                simu_i = np.concatenate((simu_i,simu), axis = 0)
                
        num_events = len(heure_i)-1
        num_data_points = 7
        Comp_simu_BLH = xr.Dataset(
            {
            'lon_comp': (['event'], lon_comp_i[1:].astype(np.float32)),
            'lat_comp': (['event'], lat_comp_i[1:].astype(np.float32)),
            'bias': (['event'], bias_i[1:].astype(np.float32)),
            'traces': (['event'], traces_i[1:].astype(np.float32)),
            'simu': (['event'], simu_i[1:].astype(np.float32)),
            'mean_topo': (['event'], bias_i[1:].astype(np.float32)),
            'hquart_topo': (['event'], hquart_topo_i[1:].astype(np.float32)),
            'std_topo': (['event'], std_i[1:].astype(np.float32)),
            'heure': (['event'], heure_i[1:].astype(int))
        },
        coords={
            'event': np.arange(num_events)
        },
        )
        
        Comp_simu_BLH.to_netcdf(savenet_path + 'Comp_'+save_name+'.nc') 

def new_scatter_th(reduced_thermal,reduced_simu,extent_son,ngridcell):
 
    dlon = (extent_son[1]-extent_son[0])/ngridcell
    dlat = (extent_son[3]-extent_son[2])/ngridcell

    max_speed_th = np.zeros((4, ngridcell, ngridcell))

    altitudes = np.arange(800,5200,200)
    
    ticks = np.arange(len(altitudes)-1)
    labels_ticks = ['['+str(altitudes[i]) +';'+ str(altitudes[i+1])+ ']' for i in range(len(altitudes)-1)]
    
    dist_speed = []
    alti_th = []
    lon_th = []
    lat_th = []
    simu_m_speed = []
    
    for l in tqdm(range(len(altitudes)-1)) :
        
        reduced_comp_alti = reduced_thermal.where(
            (reduced_thermal['altitude_gnss'] <= altitudes[l+1]) & (reduced_thermal['altitude_gnss'] >= altitudes[l]),
            drop=True) 
    
        indexes = list(set(reduced_comp_alti['index'].values))
        
        for j in indexes :
    
            a_thermal = reduced_comp_alti.where(
                (reduced_comp_alti['index'] == j),
                drop=True) 
            
            if len(a_thermal['altitude_gnss'].values) > 20 :                
    
                a_thermal_alt_sm = smooth(a_thermal['altitude_gnss'].values,window_len=20,window='hanning')
                
                mean_speed = (a_thermal_alt_sm[-1]-a_thermal_alt_sm[0])/(a_thermal['rawtime'].values[-1]-a_thermal['rawtime'].values[0])
                
                foward_alt = a_thermal_alt_sm[2:]
                backward_alt = a_thermal_alt_sm[:-2]
                foward_time = a_thermal['rawtime'].values[2:]
                backward_time = a_thermal['rawtime'].values[:-2]

                max_speed = np.max((foward_alt-backward_alt)/(foward_time-backward_time))
                
                if max_speed > 0 and max_speed < 8 :
                        
                    if a_thermal['type'].values[0] == 0 :
                        max_speed += 1.1
    
                    else :
                        max_speed += 0.6

                    lon = np.mean(a_thermal['longitude'].values)
                    lat = np.mean(a_thermal['latitude'].values)

                    dlon_simu,dlat_simu = 0.02,0.02

                    simu_a = reduced_simu.where(
                        (reduced_simu['lon'] >= lon-dlon_simu) & (reduced_simu['lon'] <= lon+dlon_simu) & 
                        (reduced_simu['lat'] >= lat-dlat_simu) & (reduced_simu['lat'] <= lat+dlat_simu),
                        drop = True)

                    indx, indy = int((lon - extent_son[0])/dlon),int((lat - extent_son[2])/dlat)

                    if 0 < indx < ngridcell and 0 < indy < ngridcell :
                        if max_speed > max_speed_th[0,indx,indy] and len(simu_a['W_max'].values) !=0 :
                            max_speed_th[0,indx,indy] = max_speed
                            max_speed_th[2,indx,indy] = (altitudes[l+1]+altitudes[l])/2
                            max_speed_th[3,indx,indy] = a_thermal['type'].values[0]+1
                            max_speed_th[1,indx,indy] = np.max(simu_a['W_max'].values)

    for i in range(ngridcell):
        for j in range(ngridcell):
            if max_speed_th[2,i,j] < 100 :
                 max_speed_th[0,i,j],max_speed_th[1,i,j],max_speed_th[2,i,j],max_speed_th[3,i,j] = np.nan, np.nan, np.nan, np.nan

    return max_speed_th

def new_diurn_cycle_th(thermals_daily,simu_daily):
    res =  np.zeros((6,24))
    for i in range(23):
        
        reduced_thermal_19_it = thermals_daily.where(
            (thermals_daily['rawtime'] >= i*60*60) & (thermals_daily['rawtime'] <= (i+1)*60*60),
            drop = True)

        if len(reduced_thermal_19_it['type']) != 0 :
            reduced_simu_it = simu_daily.where(
                (simu_daily['heure'] == i),
                drop = True)

            extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
            ngridcell = 100 
            max_speed_th = new_scatter_th(reduced_thermal_19_it,reduced_simu_it,extent_son,ngridcell)

            res[0,i] = np.nanpercentile(np.ravel(max_speed_th[0]),25)
            res[1,i] = np.nanpercentile(np.ravel(max_speed_th[0]),50)
            res[2,i] = np.nanpercentile(np.ravel(max_speed_th[0]),75)
            res[3,i] = np.nanpercentile(np.ravel(max_speed_th[1]),25)
            res[4,i] = np.nanpercentile(np.ravel(max_speed_th[1]),50)
            res[5,i] = np.nanpercentile(np.ravel(max_speed_th[1]),75)

    for i in range(23):
        for j in range(6):
            if res[j,i] == 0 :
                res[j,i] = np.nan

    return res

In [ ]:
### Comparison
dlat, dlon = 0.016,0.023 # ~ 1.8 * 1.8 km²

simu_BLH_19_son = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_19.nc')   
simu_BLH_20_son = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_20.nc')
Comp(19,'BLH',dlat,dlon,simu_BLH_19_son,'BLH_son_19')
Comp(20,'BLH',dlat,dlon,simu_BLH_19_son,'BLH_son_20')

simu_BLH_19_father = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_0_23.nc')   
simu_BLH_20_father = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_20_0_23.nc')
Comp(19,'BLH',dlat,dlon,simu_BLH_19_father,'BLH_father_19')
Comp(20,'BLH',dlat,dlon,simu_BLH_20_father,'BLH_father_20')

simu_BLH_19_AROME = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_19.nc')   
simu_BLH_20_AROME = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_20.nc')
Comp(19,'BLH',dlat,dlon,simu_BLH_19_AROME,'BLH_AROME_19')
Comp(20,'BLH',dlat,dlon,simu_BLH_20_AROME,'BLH_AROME_20')

### Diurn cycle
AROME_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_19.nc')
AROME_19_0_23_spatial = AROME_19_0_23.where(
    (AROME_19_0_23['lon'] >= extent_son[0]) & (AROME_19_0_23['lon'] <= extent_son[1]) & 
    (AROME_19_0_23['lat'] >= extent_son[2]) & (AROME_19_0_23['lat'] <= extent_son[3]),
    drop = True)
diurn_cycle_AROME_19 = new_diurn_cycle_th(reduced_thermal_19,AROME_19_0_23_spatial)

Father_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_father_19.nc')
Father_19_0_23_spatial = Father_19_0_23.where(
    (Father_19_0_23['lon'] >= extent_son[0]) & (Father_19_0_23['lon'] <= extent_son[1]) & 
    (Father_19_0_23['lat'] >= extent_son[2]) & (Father_19_0_23['lat'] <= extent_son[3]),
    drop = True)
diurn_cycle_Father_19 = new_diurn_cycle_th(reduced_thermal_19,Father_19_0_23_spatial)

Son_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_19.nc')
Son_19_0_23_spatial = Son_19_0_23.where(
    (Son_19_0_23['lon'] >= extent_son[0]) & (Son_19_0_23['lon'] <= extent_son[1]) & 
    (Son_19_0_23['lat'] >= extent_son[2]) & (Son_19_0_23['lat'] <= extent_son[3]),
    drop = True)
diurn_cycle_son_19 = new_diurn_cycle_th(reduced_thermal_19,Son_19_0_23_spatial)

## Plotting

### Functions

In [6]:
def compute_diurn_cycle_BLH(var_simu, var_sta,  percent, Comp_simu_BHL_19):
    
    mediane_sta = [[] for i in range(24)]
    mediane_simu = [[] for i in range(24)]
    
    for i in range(24) :
        comp_it_simu = Comp_simu_BHL_19.where(
        (Comp_simu_BHL_19['heure'] == i),
            drop = True)
        
        if len(comp_it_simu['heure']) != 0 :
            mediane_sta[i] = np.nanpercentile(comp_it_simu[var_sta],percent)
            mediane_simu[i] = np.nanpercentile(comp_it_simu[var_simu],percent)
        else : 
            mediane_sta[i] = np.nan
            mediane_simu[i] = np.nan

    return mediane_sta,mediane_simu

@njit()
def selecton_type_th(max_speed_th):
    a,b,c = np.shape(max_speed_th)
    ind_para = np.zeros((b,c))
    ind_plan = np.zeros((b,c))
    for i in range(b):
        for j in range(c):
            if max_speed_th[3,i,j] == 1 :
                ind_para[i,j] = 1
                ind_plan[i,j] = np.nan
            elif max_speed_th[3,i,j] == 2 :
                ind_plan[i,j] = 1
                ind_para[i,j] = np.nan
            else :
                ind_para[i,j] = np.nan
                ind_plan[i,j] = np.nan

    return ind_para, ind_plan

@njit()
def remove_plan_para(max_speed_th):
    ind_para, ind_plan = selecton_type_th(max_speed_th)
    a,b,c = np.shape(max_speed_th)
    res_para = np.zeros((3,b,c))
    res_plan = np.zeros((3,b,c))
    for i in range(b):
        for j in range(c):
            if np.isnan(ind_para[i,j]) == False :
                res_para[:,i,j] = max_speed_th[:-1,i,j]
#                res_plan[:,i,j] = np.nan
            elif np.isnan(ind_plan[i,j]) == False:
                res_plan[:,i,j] = max_speed_th[:-1,i,j]
#                res_para[:,i,j] = np.nan

    return res_para, res_plan

def scatter_th(reduced_thermal_19_12):

    indexes = list(set(reduced_thermal_19_12['index'].values))
    
    lats = []
    lons = []
    v_speed = []
    max_speeds = []
    
    for j in indexes :
        a_thermal = reduced_thermal_19_12.where(
                (reduced_thermal_19_12['index'] == j),
                drop=True) 
        
        if len(a_thermal['altitude_gnss'].values) > 60 :

            
            a_thermal_alt_sm = smooth(a_thermal['altitude_gnss'].values,window_len=60,window='hanning')
            
            mean_speed = (a_thermal_alt_sm[-1]-a_thermal_alt_sm[0])/(a_thermal['rawtime'].values[-1]-a_thermal['rawtime'].values[0])
            """
            foward_alt = a_thermal['altitude_gnss'].values[2:]
            backward_alt = a_thermal['altitude_gnss'].values[:-2]
            foward_time = a_thermal['rawtime'].values[2:]
            backward_time = a_thermal['rawtime'].values[:-2]

            max_speed = np.max((foward_alt-backward_alt)/(foward_time-backward_time))
            """
            
            foward_alt = a_thermal_alt_sm[2:]
            backward_alt = a_thermal_alt_sm[:-2]
            foward_time = a_thermal['rawtime'].values[2:]
            backward_time = a_thermal['rawtime'].values[:-2]

            max_speed = np.max((foward_alt-backward_alt)/(foward_time-backward_time))
            
            if mean_speed > -0.5 and mean_speed < 7 :
                    
                if a_thermal['type'].values[0] == 0 :
                    mean_speed += 1.1
                    max_speed += 1.1
    
                else :
                    mean_speed += 0.6
                    max_speed += 0.6
    
                latitude_a = (a_thermal['latitude'].values[-1] + a_thermal['latitude'].values[0])/2
                longitude_a = (a_thermal['longitude'].values[-1] + a_thermal['longitude'].values[0])/2
        
                lats.append(latitude_a)
                lons.append(longitude_a)
                v_speed.append(mean_speed)
                max_speeds.append(max_speed)

    return np.array(lons),np.array(lats),np.array(v_speed),np.array(max_speeds)

def plotting_para(reduced_19,Comp_simu_BLH_19_12,Comp_simu_BLH_19_6,reduced_thermal_19_12,reduced_thermal_19_15,Father_19_0_23_spatial,reduced_thermal_19,diurn_cycle_th,day,type,save_name,color_plot):
    
    fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(15,7),layout="constrained")
    
    var = ['simu','traces']
    var_simu = ['BLH','W_max']
    cmaps = ['YlGnBu','seismic']
    scatter_labels = ['BLH simulation (m asl)','BLH gliders (m asl)','W max simulation ($m.s^{-1}$)','W max gliders ($m.s^{-1}$)']
    diurn_labels = ['BLH (m asl)', 'W max ($m.s^{-1}$)']
    
    min_l = [1500,-8]
    max_l = [5000,8,]
    pas = [1000,4]
    extends = ['max','both']
    clabels = ['BLH (m asl)','W max ($m.s^{-1}$)']
    P0 = 100000
    laterals = [300,2]
    ymin = [1500,1]
    ymax = [5000,8]
    
    for i in range(2):
        for j in range(4) :
        
            ax = axs[i,j]
            ax.text(0, 1.1, string.ascii_lowercase[i*4+j], transform=ax.transAxes, 
                size=20, weight='bold')
            
            if j == 0 :
                reduced_19_12 = reduced_19.where(
                    (reduced_19['heure'] == 12),
                    drop = True)
                
                if var_simu[i] == 'BLH' :
                    c = reduced_19_12['BLH'].values
                    cax = ax.scatter(reduced_19_12['lon'],reduced_19_12['lat'], s = 1,c = c,cmap = cmaps[i],vmin = min_l[i], vmax = max_l[i])
                    ax.scatter(Comp_simu_BLH_19_12['lon_comp'],Comp_simu_BLH_19_12['lat_comp'],c = Comp_simu_BLH_19_12['traces'],cmap = cmaps[i],vmin = min_l[i], vmax = max_l[i])
    
                elif var_simu[i] == 'W_max' :
                    c = reduced_19_12['W_max']
                    cax = ax.scatter(reduced_19_12['lon'],reduced_19_12['lat'], s = 1,c = c,cmap = cmaps[i],vmin = min_l[i], vmax = max_l[i])
                    lons,lats,v_speed,max_speeds = scatter_th(reduced_thermal_19_12)
                    ax.scatter(lons,lats,c = max_speeds,cmap = cmaps[i],vmin = min_l[i], vmax = max_l[i])
                    
                ax.set_xlabel('longitude (°E)',fontsize = size)
                ax.set_ylabel('latitude (°N)',fontsize = size)

                xticks = np.arange(5.5,7.5,0.5)
                xlabels = [str(i) for i in xticks]
                yticks = np.arange(44,46,0.4)
                ylabels = [str(i) for i in yticks]

                ax.set_xticks(xticks,xlabels,fontsize = size)
                ax.set_yticks(yticks,ylabels,fontsize = size)
            
                ticks = np.arange(min_l[i],max_l[i]+pas[i],pas[i])
                
                cbar = fig.colorbar(cax,
                            ax = ax,
                            ticks=ticks,
                            format=mticker.FixedFormatter(ticks),
                            location = 'left',
                            extend=extends[i],
                            orientation='vertical',
                            fraction = 0.028,
                            aspect = 35,
                            boundaries = ticks
                            )
            
                cbar.ax.tick_params(labelsize=20) 
                cbar.ax.set_title(label = clabels[i],fontsize = size)
    #            extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
                ax.set_xlim((5.499792,7.032229))
                ax.set_ylim((44.104446,45.18888))
                ax.set_title(type + ' 12h '+str(day)+'/08',fontsize = size)
                
            if j == 2 :

                if i == 0 :
    
                    cax_BHL = ax.scatter(Comp_simu_BLH_19_12['traces'],Comp_simu_BLH_19_12['simu'],c = Comp_simu_BLH_19_12['hquart_topo'],cmap = 'terrain',marker = 'x',vmin = 500, vmax = 5500)
                    ax.plot(np.linspace(min_l[i],max_l[i],10000),np.linspace(min_l[i],max_l[i],10000),color = 'k', linestyle = 'solid')
                    ax.plot(np.linspace(min_l[i],max_l[i],10000)+laterals[i],np.linspace(min_l[i],max_l[i],10000),color = 'k', linestyle = 'dashed')
                    ax.plot(np.linspace(min_l[i],max_l[i],10000),np.linspace(min_l[i],max_l[i],10000)+laterals[i],color = 'k', linestyle = 'dashed')
        
                    ax.set_xlabel(scatter_labels[2*i+1],fontsize = size)
                    ax.set_ylabel(scatter_labels[2*i],fontsize = size)
    
                    ticks = np.arange(500,5500,1000)
                    
                    cbar = fig.colorbar(cax_BHL,
                                ax = axs[i,1:-1],
                                ticks=ticks,
                                format=mticker.FixedFormatter(ticks),
                                location = 'top',
                                extend='max',
                                orientation='horizontal',
                                fraction = 0.04,
                                aspect = 65,
                                boundaries = ticks
                                )
                
                    cbar.ax.tick_params(labelsize=size) 
                    cbar.ax.set_title(label = '$90^{th}$ q alt (m asl)',fontsize = size)

                ax.set_xlim((min_l[i],max_l[i]))
                ax.set_ylim((min_l[i],max_l[i]))

                ticks_w = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                labels_w = [str(i) for i in ticks_w]

                ax.set_xticks(ticks_w,labels_w,fontsize = size)
                ax.set_yticks(ticks_w,labels_w,fontsize = size)

                ax.set_title('15h '+str(day)+'/08')
    
                if i == 1 :

                    simu_19_15 = reduced_19.where(
                        (reduced_19['heure'] == 15),
                        drop = True)
                    
                    ngridcell = 100
                    extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
                    max_speed_th = new_scatter_th(reduced_thermal_19_15,simu_19_15,extent_son,ngridcell)
                    res_para, res_plan = remove_plan_para(max_speed_th)
                    
                    cax_th = ax.scatter(np.ravel(res_para[0]),np.ravel(res_para[1]), c = np.ravel(res_para[2]), cmap = 'terrain',marker = 'o',alpha = 0.8, vmin = 500, vmax = 5500, edgecolor = 'k')
                    ax.scatter(np.ravel(res_plan[0]),np.ravel(res_plan[1]), c = np.ravel(res_plan[2]), cmap = 'terrain',marker = 'o',alpha = 0.8, vmin = 0, vmax = 4500)

                    ax.plot(np.linspace(1,11,10000),np.linspace(1,11,10000),color = 'k', linestyle = 'solid')
                    ax.plot(np.linspace(1,11,10000)+laterals[i],np.linspace(1,11,10000),color = 'k', linestyle = 'dashed')
                    ax.plot(np.linspace(1,11,10000),np.linspace(1,11,10000)+laterals[i],color = 'k', linestyle = 'dashed')

                    ax.scatter([-10,-10],[-10,-10],marker = 'o', color = 'y', edgecolor = 'k', label = 'Paragliding')
                    ax.scatter([-10,-10],[-10,-10],marker = 'o', color = 'y', label = 'Gliding')
        
                    ax.set_xlabel('Thermal W max ($m.s^{-1}$)',fontsize = size)
                    ax.set_ylabel('Simu W max($m.s^{-1}$)',fontsize = size)
                    ticks = np.arange(500,5500,1000)
                    """
                    cbar = fig.colorbar(cax_th,
                                ax = axs[i,j],
                                ticks=ticks,
                                format=mticker.FixedFormatter(ticks),
                                location = 'bottom',
                                extend='max',
                                orientation='horizontal',
                                fraction = 0.028,
                                aspect = 35,
                                boundaries = ticks
                                )
                
                    cbar.ax.tick_params(labelsize=size) 
                    cbar.ax.set_title(label = 'Cirlcing up elevation', fontsize=size)
                    """
                    ax.set_xlim((1,11))
                    ax.set_ylim((1,11))

                    ticks_w = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                    labels_w = [str(i) for i in ticks_w]

                    ax.set_xticks(ticks_w,labels_w,fontsize = size)
                    ax.set_yticks(ticks_w,labels_w,fontsize = size)

                    ax.legend()

                ax.set_title('15h '+str(day)+'/08',fontsize = size)

            if j == 1 :
    
                if i == 0 :


                    cax_BHL = ax.scatter(Comp_simu_BLH_19_6['traces'],Comp_simu_BLH_19_6['simu'],c = Comp_simu_BLH_19_6['hquart_topo'],cmap = 'terrain',marker = 'x',vmin = 500, vmax = 5500)
                    ax.plot(np.linspace(min_l[i],max_l[i],10000),np.linspace(min_l[i],max_l[i],10000),color = 'k', linestyle = 'solid')
                    ax.plot(np.linspace(min_l[i],max_l[i],10000)+laterals[i],np.linspace(min_l[i],max_l[i],10000),color = 'k', linestyle = 'dashed')
                    ax.plot(np.linspace(min_l[i],max_l[i],10000),np.linspace(min_l[i],max_l[i],10000)+laterals[i],color = 'k', linestyle = 'dashed')
        
                    ax.set_xlabel(scatter_labels[2*i+1],fontsize = size)
                    ax.set_ylabel(scatter_labels[2*i],fontsize = size)
                    ticks = np.arange(500,5500,1000)
                    """
                    cbar = fig.colorbar(cax_BHL,
                                ax = axs[i,1:-1],
                                ticks=ticks,
                                format=mticker.FixedFormatter(ticks),
                                location = 'top',
                                extend='max',
                                orientation='horizontal',
                                fraction = 0.028,
                                aspect = 35,
                                boundaries = ticks
                                )
                
                    cbar.ax.tick_params(labelsize=size) 
                    cbar.ax.set_title(label = '$90^{th}$ q alt (m asl)', fontsize=size)
                    """
                    ax.set_xlim((min_l[i],max_l[i]))
                    ax.set_ylim((min_l[i],max_l[i]))

                    ticks_w = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                    labels_w = [str(i) for i in ticks_w]

                    ax.set_xticks(ticks_w,labels_w,fontsize = size)
                    ax.set_yticks(ticks_w,labels_w,fontsize = size)

                if i == 1 :
                    ngridcell = 100
                    extent_son = (5.499792, 7.032229, 44.104446, 45.18888)

                    simu_19_12 = reduced_19.where(
                        (reduced_19['heure'] == 12),
                        drop = True)

                    max_speed_th = new_scatter_th(reduced_thermal_19_12,simu_19_12,extent_son,ngridcell)
                    res_para, res_plan = remove_plan_para(max_speed_th)
                    
                    cax_th = ax.scatter(np.ravel(res_para[0]),np.ravel(res_para[1]), c = np.ravel(res_para[2]), cmap = 'terrain', alpha = 0.8, marker = 'o',vmin = 500, vmax = 5500, edgecolor = 'k')
                    ax.scatter(np.ravel(res_plan[0]),np.ravel(res_plan[1]), c = np.ravel(res_plan[2]), cmap = 'terrain',alpha = 0.8, marker = 'o',vmin = 0, vmax = 4500)

                    ax.plot(np.linspace(1,11,10000),np.linspace(1,11,10000),color = 'k', linestyle = 'solid')
                    ax.plot(np.linspace(1,11,10000)+laterals[i],np.linspace(1,11,10000),color = 'k', linestyle = 'dashed')
                    ax.plot(np.linspace(1,11,10000),np.linspace(1,11,10000)+laterals[i],color = 'k', linestyle = 'dashed')

                    ax.scatter([-10,-10],[-10,-10],marker = 'o', color = 'y', edgecolor = 'k', label = 'Paragliding')
                    ax.scatter([-10,-10],[-10,-10],marker = 'o', color = 'y', label = 'Gliding')
        
                    ax.set_xlabel('Thermal W max ($m.s^{-1}$)',fontsize = size)
                    ax.set_ylabel('Simu W max ($m.s^{-1}$)',fontsize = size)
                    ticks = np.arange(500,5500,1000)
                    
                    cbar = fig.colorbar(cax_th,
                                ax = axs[i,1:-1],
                                ticks=ticks,
                                format=mticker.FixedFormatter(ticks),
                                location = 'bottom',
                                extend='max',
                                orientation='horizontal',
                                fraction = 0.04,
                                aspect = 65,
                                boundaries = ticks
                                )
                
                    cbar.ax.tick_params(labelsize=size) 
                    cbar.ax.set_title(label = 'Cirlcing up elevation (m asl)', fontsize=size)

                    ax.set_xlim((1,11))
                    ax.set_ylim((1,11))
                    ax.legend()

                    ticks_w = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                    labels_w = [str(i) for i in ticks_w]

                    ax.set_xticks(ticks_w,labels_w,fontsize = size)
                    ax.set_yticks(ticks_w,labels_w,fontsize = size)

                ax.set_title('12h '+str(day)+'/08',fontsize = size)

            if j == 3 :
    
                ticks_time = np.array([i for i in range(0,24,3)])
                labels_ticks_time = [str(i)+'h' for i in ticks_time]

                ax.grid(visible=True, axis='both', color = '0.80')
                ax.set_xticks(ticks_time, labels_ticks_time, fontsize=size*0.9)

                if i == 0 :
                    
                    ax.fill_between(np.arange(0,24,1), lows[1,:], highs[1,:],color = color_plot,alpha = 0.3)
                    ax.fill_between(np.arange(0,24,1), lows[0,:], highs[0,:],color = 'k',alpha = 0.3)
                    ax.plot(np.arange(0,24,1), medians[0,:], linestyle ='solid', color = 'k',label ='Flights')
                    ax.plot(np.arange(0,24,1), medians[1,:], linestyle ='solid', color = color_plot,label = 'Simulation')
                    
                    """
                    ax.plot(medians[0,:],'k',label ='Flights')
                    ax.plot(medians[1,:],'r',label = 'Simulation')
                    ax.plot(lows[0,:],linestyle = 'dashed',color = 'k')
                    ax.plot(lows[1,:],linestyle = 'dashed',color = 'r')
                    ax.plot(highs[0,:],linestyle = 'dashed',color = 'k')
                    ax.plot(highs[1,:],linestyle = 'dashed',color = 'r')
                    """
                    ax.vlines(12, ymin = 2500, ymax = 4500, linestyle = 'dashed', color  ='r')
                    ax.vlines(15, ymin = 2500, ymax = 4500, linestyle = 'dashed', color  ='r')

                    ax.set_title(type + ' ' +str(day)+'/08',fontsize = size)
                    yticks = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                    ylabels = [str(i) for i in yticks]

                if i == 1 :

                    line_styles = ['dashed','solid','dashed']

                    #diurn_th_19 = diurn_cycle_th(Father_19_0_23_spatial,reduced_thermal_19)
                    #diurn_cycle_th = new_diurn_cycle_th(reduced_thermal_19_12,reduced_son_19_12)
                    
                    ax.fill_between(np.arange(0,24,1), diurn_cycle_th[3,:], diurn_cycle_th[5,:],color = color_plot,alpha = 0.3)
                    ax.fill_between(np.arange(0,24,1), diurn_cycle_th[0,:], diurn_cycle_th[2,:],color = 'k',alpha = 0.3)
                    ax.plot(np.arange(0,24,1), diurn_cycle_th[1,:], linestyle =line_styles[1], color = 'k')
                    ax.plot(np.arange(0,24,1), diurn_cycle_th[4,:], linestyle =line_styles[1], color = color_plot)

                    """
                    for l in range(3):
                        ax.plot(diurn_cycle_th[3+l,:], linestyle =line_styles[l], color = 'r')
                        ax.plot(diurn_cycle_th[l,:], linestyle =line_styles[l], color = 'k')
                    """

                    ax.plot([-10,-10],[-10,-10],color = 'k',label = 'Flights')
                    ax.plot([-10,-10],[-10,-10],color = color_plot,label = 'Simulation')
                    ax.set_ylim((0,10))

                    ax.vlines(12, ymin = -1, ymax = 11, linestyle = 'dashed', color  ='r')
                    ax.vlines(15, ymin = -1, ymax = 11, linestyle = 'dashed', color  ='r')

                    ax.set_title(type + ' ' +str(day)+'/08')
                    yticks = np.arange(ymin[i],ymax[i]+pas[i],pas[i])
                    ylabels = [str(i) for i in yticks]
                    
                ax.set_ylabel(diurn_labels[i],fontsize = size)

                ax.set_yticks(yticks,ylabels,fontsize = size)
                ax.set_xlim((0,24))
                ax.legend()

    plt.savefig(savefig_path+save_name+'.png')

def plotting_time_series_quantils(netcdf,vars):
    medians = np.zeros((2,24))
    lows = np.zeros((2,24))
    highs = np.zeros((2,24))
    for i in range(1) :
        medians[i,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],50,netcdf)[0]
        medians[i+1,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],50,netcdf)[1]
    
        highs[i,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],75,netcdf)[0]
        highs[i+1,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],75,netcdf)[1]
    
        lows[i,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],25,netcdf)[0]
        lows[i+1,:] = compute_diurn_cycle_BLH(var[i], vars[i+1],25,netcdf)[1]

    return lows,medians,highs
    


### Loading products

In [ ]:
### AROME
Comp_simu_BLH_19_AROME = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS//Netcdfffs/true_comp_BLH/Comp_BLH_AROME_19_BLH_.nc') 

Comp_simu_BLH_19_6_AROME = Comp_simu_BLH_19_AROME.where(
    (Comp_simu_BLH_19_AROME['heure'] == 12),
    drop = True)

Comp_simu_BLH_19_12_AROME = Comp_simu_BLH_19_AROME.where(
    (Comp_simu_BLH_19_AROME['heure'] == 15),
    drop = True)

AROME_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_AROME_19.nc')

AROME_19_0_23_spatial = AROME_19_0_23.where(
    (AROME_19_0_23['lon'] >= extent_son[0]) & (AROME_19_0_23['lon'] <= extent_son[1]) & 
    (AROME_19_0_23['lat'] >= extent_son[2]) & (AROME_19_0_23['lat'] <= extent_son[3]),
    drop = True)

reduced_AROME_19_12 = AROME_19_0_23_spatial.where(
    (AROME_19_0_23_spatial['heure'] == 12),
    drop = True)

### Father
Comp_simu_BLH_19_father = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS//Netcdfffs/true_comp_BLH/Comp_BLH_father_19_BLH_.nc') 

Comp_simu_BLH_19_6_father = Comp_simu_BLH_19_father.where(
    (Comp_simu_BLH_19_father['heure'] == 12),
    drop = True)

Comp_simu_BLH_19_12_father = Comp_simu_BLH_19_father.where(
    (Comp_simu_BLH_19_father['heure'] == 15),
    drop = True)

father_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/father_0_23.nc')

Father_19_0_23_spatial = father_19_0_23.where(
    (father_19_0_23['lon'] >= extent_son[0]) & (father_19_0_23['lon'] <= extent_son[1]) & 
    (father_19_0_23['lat'] >= extent_son[2]) & (father_19_0_23['lat'] <= extent_son[3]),
    drop = True)

reduced_Father_19_12 = Father_19_0_23_spatial.where(
    (Father_19_0_23_spatial['heure'] == 12),
    drop = True)

### Son
Comp_simu_BLH_19_son = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS//Netcdfffs/true_comp_BLH/Comp_BLH_son_19_BLH_.nc') 

Comp_simu_BLH_19_12_son = Comp_simu_BLH_19_son.where(
    (Comp_simu_BLH_19_son['heure'] == 12),
    drop = True)

Comp_simu_BLH_19_15_son = Comp_simu_BLH_19_son.where(
    (Comp_simu_BLH_19_son['heure'] == 15),
    drop = True)

son_19_0_23 = xr.open_dataset('T:/C2H/STAGES/LEO_BARROIS/Netcdfffs/BHL_W_max_son_19.nc')

Son_19_0_23_spatial = son_19_0_23.where( #reduced_Son_19
    (son_19_0_23['lon'] >= extent_son[0]) & (son_19_0_23['lon'] <= extent_son[1]) & 
    (son_19_0_23['lat'] >= extent_son[2]) & (son_19_0_23['lat'] <= extent_son[3]),
    drop = True)

reduced_Son_19_12 = Son_19_0_23_spatial.where(
    (Son_19_0_23_spatial['heure'] == 12),
    drop = True)

### AROME

In [ ]:
vars = ['simu','traces']
lows,medians,highs = plotting_time_series_quantils(Comp_simu_BLH_19_AROME,vars)

size = 15
extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
plotting_para(AROME_19_0_23_spatial,Comp_simu_BLH_19_12_AROME,Comp_simu_BLH_19_6_AROME,reduced_thermal_19_12,reduced_thermal_19_15,AROME_19_0_23_spatial,reduced_thermal_19,diurn_cycle_AROME_19,19,'AROME','BHL_AROME_19','r')

### Meso-NH 600

In [ ]:
vars = ['simu','traces']
lows,medians,highs = plotting_time_series_quantils(Comp_simu_BLH_19_father,vars)

size = 15
extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
plotting_para(Father_19_0_23_spatial,Comp_simu_BLH_19_12_father,Comp_simu_BLH_19_6_father,reduced_thermal_19_12,reduced_thermal_19_15,Father_19_0_23_spatial,reduced_thermal_19,diurn_cycle_father_19,19,'LES600','BHL_father_19','g')

### Mes-NH 200

In [ ]:
vars = ['simu','traces']
lows,medians,highs = plotting_time_series_quantils(Comp_simu_BLH_19_son,vars)

size = 15
extent_son = (5.499792, 7.032229, 44.104446, 45.18888)
plotting_para(Son_19_0_23_spatial,Comp_simu_BLH_19_12_son,Comp_simu_BLH_19_6_son,reduced_thermal_19_12,reduced_thermal_19_15,Son_19_0_23_spatial,reduced_thermal_19,diurn_cycle_son_19,19,'LES200','BHL_son_19','b')